In [1]:
from utils import *

train_with_pytorch_lightning = True

s2s_args = ArgumentsS2S(
    batch_size= 16,
    max_length=512
)

path_train = "C:/Users/Admin/Downloads/ELI5.jsonl"
path_val = "C:/Users/Admin/Downloads/ELI5_val.jsonl"

# dataset_train = load_dataset('json', data_files = path_train)
dataset_val = load_dataset('json', data_files = path_val)

# train = dataset_train['train']
val = dataset_val['train']

C:\Users\Admin\anaconda3\envs\env\lib\site-packages\torchaudio\backend\utils.py:74: UserWarning: No audio backend is available.
  warnings.warn("No audio backend is available.")
Using custom data configuration default-2dd2a5479f3fb3f2
Reusing dataset json (C:\Users\Admin\.cache\huggingface\datasets\json\default-2dd2a5479f3fb3f2\0.0.0\ac0ca5f5289a6cf108e706efcf040422dbbfa8e658dee6a819f20d76bb84d26b)


  0%|          | 0/1 [00:00<?, ?it/s]

In [2]:
checkpoint = "t5-base"
model = T5ForConditionalGeneration.from_pretrained(checkpoint)


In [3]:
tokenizer = T5TokenizerFast.from_pretrained(checkpoint)

C:\Users\Admin\anaconda3\envs\env\lib\site-packages\transformers\models\t5\tokenization_t5_fast.py:155: FutureWarning: This tokenizer was incorrectly instantiated with a model max length of 512 which will be corrected in Transformers v5.
For now, this behavior is kept to avoid breaking backwards compatibility when padding/encoding with `truncation is True`.
- Be aware that you SHOULD NOT rely on t5-base automatically truncating your input to 512 when padding/encoding.
- If you want to encode/pad to sequences longer than 512 you can either instantiate this tokenizer with `model_max_length` or pass `max_length` when encoding/padding.
- To avoid this warning, please instantiate this tokenizer with `model_max_length` set to your preferred value.
  warnings.warn(


In [4]:
# train1 = train.map(lambda ex: preprocess(ex,num_docs=5), remove_columns = ['question_id'])
val1 = val.map(lambda ex: preprocess(ex,num_docs=5), remove_columns = ['question_id'])

Parameter 'function'=<function <lambda> at 0x000001DB84EA0CA0> of the transform datasets.arrow_dataset.Dataset._map_single couldn't be hashed properly, a random hash was used instead. Make sure your transforms and parameters are serializable with pickle or dill for the dataset fingerprinting and caching to work. If you reuse this transform, the caching mechanism will consider it to be different from the previous calls and recompute everything. This warning is only showed once. Subsequent hashing failures won't be showed.


  0%|          | 0/1507 [00:00<?, ?ex/s]

In [5]:
tokenizer.sep_token = '<sep>'

In [6]:
tokenizer.add_tokens(['<sep>'])

1

In [7]:
len(tokenizer)

32101

In [8]:
tokenizer

T5TokenizerFast(name_or_path='t5-base', vocab_size=32100, model_max_length=512, is_fast=True, padding_side='right', truncation_side='right', special_tokens={'eos_token': '</s>', 'unk_token': '<unk>', 'sep_token': '<sep>', 'pad_token': '<pad>', 'additional_special_tokens': ['<extra_id_0>', '<extra_id_1>', '<extra_id_2>', '<extra_id_3>', '<extra_id_4>', '<extra_id_5>', '<extra_id_6>', '<extra_id_7>', '<extra_id_8>', '<extra_id_9>', '<extra_id_10>', '<extra_id_11>', '<extra_id_12>', '<extra_id_13>', '<extra_id_14>', '<extra_id_15>', '<extra_id_16>', '<extra_id_17>', '<extra_id_18>', '<extra_id_19>', '<extra_id_20>', '<extra_id_21>', '<extra_id_22>', '<extra_id_23>', '<extra_id_24>', '<extra_id_25>', '<extra_id_26>', '<extra_id_27>', '<extra_id_28>', '<extra_id_29>', '<extra_id_30>', '<extra_id_31>', '<extra_id_32>', '<extra_id_33>', '<extra_id_34>', '<extra_id_35>', '<extra_id_36>', '<extra_id_37>', '<extra_id_38>', '<extra_id_39>', '<extra_id_40>', '<extra_id_41>', '<extra_id_42>', '<ext

In [9]:
model.resize_token_embeddings(len(tokenizer))

Embedding(32101, 768)

In [10]:
max_input_length = 512
max_target_length = 64
def add_sep_token(example):
    example['answers'] = " <sep> ".join(example['answers'])
    return example
def add_eos_token(example):
    example['ques_context'] =f"question: {example['question']} context: {example['ctxs']} </s>"
    example['answers']=  example['answers'] + " </s>"
    return example

def convert_to_features(example_batch):

    input_encodings = tokenizer.batch_encode_plus(example_batch['ques_context'],
                                                  max_length=max_input_length,
                                                  add_special_tokens=True,
                                                  truncation=True,
                                                  pad_to_max_length=True)

    target_encodings = tokenizer.batch_encode_plus(example_batch['answers'],
                                                   max_length=max_target_length,
                                                   add_special_tokens=True,
                                                   truncation=True, pad_to_max_length=True)

    encodings = {
        'input_ids': input_encodings['input_ids'],
        'attention_mask': input_encodings['attention_mask'],
        'decoder_input_ids': target_encodings['input_ids']
        ,'decoder_attention_mask': target_encodings['attention_mask']
    }

    return encodings

    

In [11]:
train2 = val1.map(add_sep_token)

  0%|          | 0/1507 [00:00<?, ?ex/s]

In [12]:
train3 = train2.map(add_eos_token)

  0%|          | 0/1507 [00:00<?, ?ex/s]

In [13]:
train3

Dataset({
    features: ['question', 'answers', 'ctxs', 'ques_context'],
    num_rows: 1507
})

In [14]:
train4 = train3.map(convert_to_features, batched=True, remove_columns=['question','answers','ctxs','ques_context'])

  0%|          | 0/2 [00:00<?, ?ba/s]

C:\Users\Admin\anaconda3\envs\env\lib\site-packages\transformers\tokenization_utils_base.py:2377: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(


In [15]:
from transformers import (
    T5ForConditionalGeneration,
    T5Tokenizer,
    EvalPrediction,
    DataCollator,
    Trainer,
    TrainingArguments)
from typing import Dict, List, Optional

import dataclasses
from dataclasses import dataclass, field

In [16]:
columns = ['input_ids', 'decoder_input_ids', 'attention_mask', 'decoder_attention_mask']
train4.set_format(type='torch', columns=columns)

In [17]:
class T2TDataCollator():
  def __call__(self, batch: List) -> Dict[str, torch.Tensor]:
    """
    Take a list of samples from a Dataset and collate them into a batch.
    Returns:
    A dictionary of tensors
    """

    input_ids = torch.stack([example['input_ids'] for example in batch])
    lm_labels = torch.stack([example['decoder_input_ids'] for example in batch])
    lm_labels[lm_labels[:, :] == 0] = -100
    attention_mask = torch.stack([example['attention_mask'] for example in batch])
    decoder_attention_mask = torch.stack([example['decoder_attention_mask'] for example in batch])

    return {
        'input_ids': input_ids,
        'attention_mask': attention_mask,
        'labels': lm_labels,
        'decoder_attention_mask': decoder_attention_mask
    }

In [18]:
training_args = TrainingArguments(output_dir="model/",
                                  per_device_train_batch_size=4,
                                  per_device_eval_batch_size=4,
                                  gradient_accumulation_steps=16,
                                  learning_rate=1e-4,
                                  num_train_epochs=7,
                                  logging_steps=100,
                                  run_name="QA_question_answering",
                                  evaluation_strategy="steps",
                                  save_steps=500,
                                  optim='adamw_torch'
                                  )

In [19]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train4,
#     eval_dataset=valid_dataset,
    data_collator=T2TDataCollator()
)

# Training
trainer.train()

C:\Users\Admin\anaconda3\envs\env\lib\site-packages\transformers\optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Step,Training Loss,Validation Loss



KeyboardInterrupt

